In [112]:
import os
from joblib import dump, load
import pandas as pd
from sklearn import model_selection
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import QuantileTransformer


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(seed=42)


def read_data(train_file, test_file, target_file, path='D:\\Dataset\\MOA\\',):
    
    df_train=pd.read_csv(os.path.join(path, train_file))
    df_test=pd.read_csv(os.path.join(path, test_file))
    df_target=pd.read_csv(os.path.join(path, target_file))
    
    print(df_train.shape, df_test.shape, df_target.shape, df_test.shape)
    
    return df_train, df_test, df_target, df_test


def process_data(train, test, target):
    df1 = train[train.cp_type!='ctl_vehicle'].reset_index(drop=True)
    df2 = test[test.cp_type!='ctl_vehicle'].reset_index(drop=True)
    
    #Handle categorical variables
    con_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    con_df = pd.get_dummies(con_df, columns=['cp_type','cp_time','cp_dose']) 
    
    #Train test splitter
    train_df = con_df.iloc[:df1.shape[0],:]
    test_df   = con_df.iloc[df1.shape[0]:,:]
    
    #Train data combiner
    final_train = pd.merge(train_df, target, on='sig_id', how='left')

    
    print(final_train.shape)
    
    return final_train, test_df


def create_folds(train, target, path='D:\\Dataset\\MOA\\'):
    df=train.copy()
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    y = target.values
    skf = MultilabelStratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    for fold,(idxT,idxV) in enumerate(skf.split(X=df, y=target)):
        df.loc[idxV, "kfold"] = fold
    print(df.kfold.value_counts())
    df.to_csv(os.path.join(path, "train_folds.csv"), index=False)
    

def Multi_log_loss(y_true, y_pred):

    score = []
    for i in range(y_true.shape[1]):
        
        log_score = metrics.log_loss(y_true[:,i].astype(float) , 
                                     y_pred[:,i].astype(float),
                                     labels=sorted(np.unique(y_true))
                                    
                                    )
        score.append(log_score)  
        
    return np.mean(np.array(score))

In [113]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [114]:
train_features, test_features, train_targets_scored, test_final = read_data('train_features.csv', 'test_features.csv', 'train_targets_scored.csv')

(23814, 876) (3982, 876) (23814, 207) (3982, 876)


In [115]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [116]:
transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
transformer.fit(train_features.loc[:, GENES + CELLS])
train_features[GENES + CELLS] = transformer.transform(train_features.loc[:, GENES + CELLS])

dump(transformer, 'rank_1858_5fold_sub.bin', compress=True)


test_features[GENES + CELLS] = transformer.transform(test_features.loc[:, GENES + CELLS])

In [117]:
    from sklearn.decomposition import PCA
    from joblib import dump, load
    # GENES
    n_comp = 100

    data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
    #data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
    
    pca_genes = PCA(n_components=n_comp, random_state=42)
    data2 = pca_genes.fit_transform(data[GENES])

    dump(pca_genes, '1858_5fold_genes_sub.bin', compress=True)
    
    train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

    train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
    test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

    # drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
    train_features = pd.concat((train_features, train2), axis=1)
    test_features = pd.concat((test_features, test2), axis=1)


In [118]:
    #CELLS
    n_comp = 15

    data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
    #data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
    
    pca_cells = PCA(n_components=n_comp, random_state=42)
    data2 = pca_cells.fit_transform(data[CELLS])
    
    dump(pca_cells, '1858_5fold_cells_sub.bin', compress=True)
    
    train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

    train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
    test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

    # drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
    train_features = pd.concat((train_features, train2), axis=1)
    test_features = pd.concat((test_features, test2), axis=1)


In [119]:
    from sklearn.feature_selection import VarianceThreshold
    var_thresh = VarianceThreshold(threshold=0.5)
    data = train_features.append(test_features)
    data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

    dump(var_thresh, 'Variance_transform_1858_sub.bin', compress=True)
    
    train_features_transformed = data_transformed[ : train_features.shape[0]]
    test_features_transformed = data_transformed[-test_features.shape[0] : ]


    train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

    train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


    test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

    test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

In [120]:
train = train_features.merge(train_targets_scored, on='sig_id')
drug = pd.read_csv('D://Dataset//MOA//train_drug.csv')
train = train.merge(drug, on='sig_id')


train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [121]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [122]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [123]:
def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    #data.loc[:, 'cp_time'] = data.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
    #data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})

# --------------------- Normalize ---------------------
#     for col in GENES:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#     for col in CELLS:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#--------------------- Removing Skewness ---------------------
#     for col in GENES + CELLS:
#         if(abs(data[col].skew()) > 0.75):
            
#             if(data[col].skew() < 0): # neg-skewness
#                 data[col] = data[col].max() - data[col] + 1
#                 data[col] = np.sqrt(data[col])
            
#             else:
#                 data[col] = np.sqrt(data[col])
    
    return data

In [124]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

def create_folds(num_starts, num_splits, trn):

    folds = []
    scored = trn.copy()
    #targets_ = scored.loc[:, train_targets_scored.columns].columns[1:].tolist()
    #train_cols = train_features.columns.tolist() + ['fold','drug_id']
    #train_cols = [col for col in train_cols if col!='cp_type']
    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index.sort_values()
    vc2 = vc.loc[vc > 18].index.sort_values()
    
    for seed in range(num_starts):
    
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}; dct2 = {}
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.groupby('drug_id')[target_cols].mean().loc[vc1]
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
            dd = {k:fold for k in tmp.index[idxV].values}
            dct1.update(dd)
    
        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop = True)
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
            dd = {k:fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)
    
        # ASSIGN FOLDS
        scored['fold'] = scored.drug_id.map(dct1)
        scored.loc[scored.fold.isna(),'fold'] =\
            scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
        scored.fold = scored.fold.astype('int8')
        folds.append(scored.fold.values)
        #train_cols = train_feats.columns.tolist() + ['fold','drug_id']
        #train_feats_main = train_feats_.merge(scored, on='sig_id', how='left')
        #train_feats_main['fold'] = train_feats_main['fold'].astype(int)
        
    return scored

In [125]:
folds= create_folds(1, 10, train)

In [126]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)

(21948, 1197)
(21948, 1198)
(3624, 990)
(21948, 207)


In [127]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [128]:
import torch.nn as nn
import torch

def train_fn(model, optimizer,scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds


In [129]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """
    def __init__(self, sigma=0.5, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.register_buffer('noise', torch.tensor(0))

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.expand(*x.size()).float().normal_() * scale
            x = x + sampled_noise
        return x 



In [130]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, dropout):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_features, hidden_size)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout)
        self.dense2 = nn.Linear(hidden_size, hidden_size)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout)
        self.dense3 = nn.Linear(hidden_size, num_targets)
        #bias_ = torch.Tensor(bias)
        #print(self.dense3.bias.data)
        #nn.init.xavier_normal_(self.dense3.weight)
        #self.dense3.bias = torch.nn.Parameter(_bias)
        self.gausian = GaussianNoise()
        #self.relu=nn.PReLU()


    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        #x = self.gausian(x)
        x = F.relu(self.dense1(x))
        x = self.gausian(x)
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [131]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['fold','sig_id','drug_id']]
len(feature_cols)

992

In [68]:
dump(target_cols, '1858_5fold_target_cols_sub.bin')
dump(feature_cols, '1858_5fold_feature_cols_sub.bin')

['1858_5fold_feature_cols_sub.bin']

In [132]:
num_features=len(feature_cols)
num_targets=len(target_cols)

In [133]:
import torch 
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 60
BATCH_SIZE = 128
LEARNING_RATE = 2.629318926022435e-05
WEIGHT_DECAY = 1e-5
NFOLDS = 10
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False


hidden_size=1400

In [134]:
import numpy as np

def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    #print(train_df[feature_cols].head())
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    #a = -np.log(y_train.mean(axis=0))
    #print(torch.Tensor(a))
    
    model = Model(
        num_features = num_features,
        num_targets = num_targets,
        hidden_size = 1400,
        dropout = 0.10721077648722396
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)

    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"D:\\MOA_Pretrained_Models\\1858_10folds _moa_sub\\{fold}_{seed}.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features = num_features,
        num_targets = num_targets,
        hidden_size = 1400,
        dropout = 0.10721077648722396
    )
    
    model.load_state_dict(torch.load(f"D:\\MOA_Pretrained_Models\\1858_10folds _moa_sub\\{fold}_{seed}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), len(target_cols)))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return   oof, predictions

In [135]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [136]:
SEED = [0, 1, 2, 3, 4, 5]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

FOLD: 0, EPOCH: 0, train_loss: 0.6755095791432165
FOLD: 0, EPOCH: 0, valid_loss: 0.36395814488915834
FOLD: 0, EPOCH: 1, train_loss: 0.06094682125314589
FOLD: 0, EPOCH: 1, valid_loss: 0.02016482567962478
FOLD: 0, EPOCH: 2, train_loss: 0.023060176341283704
FOLD: 0, EPOCH: 2, valid_loss: 0.01964694940868546
FOLD: 0, EPOCH: 3, train_loss: 0.02251327498064887
FOLD: 0, EPOCH: 3, valid_loss: 0.022578860578291556
FOLD: 0, EPOCH: 4, train_loss: 0.022465917203695545
FOLD: 0, EPOCH: 4, valid_loss: 0.019615361467003822
FOLD: 0, EPOCH: 5, train_loss: 0.02134100139862107
FOLD: 0, EPOCH: 5, valid_loss: 0.018166171058135873
FOLD: 0, EPOCH: 6, train_loss: 0.020569118316615783
FOLD: 0, EPOCH: 6, valid_loss: 0.017978368874858406
FOLD: 0, EPOCH: 7, train_loss: 0.020363950765421315
FOLD: 0, EPOCH: 7, valid_loss: 0.01795531710719361
FOLD: 0, EPOCH: 8, train_loss: 0.020342902118159877
FOLD: 0, EPOCH: 8, valid_loss: 0.018154084572897237
FOLD: 0, EPOCH: 9, train_loss: 0.020383070565519794
FOLD: 0, EPOCH: 9, va

FOLD: 1, EPOCH: 19, train_loss: 0.02081239353264532
FOLD: 1, EPOCH: 19, valid_loss: 0.018581001553684473
FOLD: 1, EPOCH: 20, train_loss: 0.020816698682404332
FOLD: 1, EPOCH: 20, valid_loss: 0.01851999040486084
FOLD: 1, EPOCH: 21, train_loss: 0.02077836972330847
FOLD: 1, EPOCH: 21, valid_loss: 0.01834105596774154
FOLD: 1, EPOCH: 22, train_loss: 0.020793574087081417
FOLD: 1, EPOCH: 22, valid_loss: 0.018584174052294757
FOLD: 1, EPOCH: 23, train_loss: 0.02075691372156143
FOLD: 1, EPOCH: 23, valid_loss: 0.018663116834229894
FOLD: 1, EPOCH: 24, train_loss: 0.02074945077540413
FOLD: 1, EPOCH: 24, valid_loss: 0.01859367209383183
FOLD: 1, EPOCH: 25, train_loss: 0.02078195781477036
FOLD: 1, EPOCH: 25, valid_loss: 0.018536794671995774
FOLD: 1, EPOCH: 26, train_loss: 0.020693598038727237
FOLD: 1, EPOCH: 26, valid_loss: 0.018677412014868524
FOLD: 1, EPOCH: 27, train_loss: 0.020706383355202213
FOLD: 1, EPOCH: 27, valid_loss: 0.018505439007033903
FOLD: 1, EPOCH: 28, train_loss: 0.020637092607155922
F

FOLD: 2, EPOCH: 38, train_loss: 0.019881325239135373
FOLD: 2, EPOCH: 38, valid_loss: 0.017589439327518146
FOLD: 2, EPOCH: 39, train_loss: 0.019743510083325446
FOLD: 2, EPOCH: 39, valid_loss: 0.017416614211267896
FOLD: 2, EPOCH: 40, train_loss: 0.01958864872013369
FOLD: 2, EPOCH: 40, valid_loss: 0.017400235920730565
FOLD: 2, EPOCH: 41, train_loss: 0.01944984848941526
FOLD: 2, EPOCH: 41, valid_loss: 0.017473640023834176
FOLD: 2, EPOCH: 42, train_loss: 0.019428580795084277
FOLD: 2, EPOCH: 42, valid_loss: 0.01747745011622707
FOLD: 2, EPOCH: 43, train_loss: 0.019277814379142177
FOLD: 2, EPOCH: 43, valid_loss: 0.01727072284039524
FOLD: 2, EPOCH: 44, train_loss: 0.019142266236726314
FOLD: 2, EPOCH: 44, valid_loss: 0.01734085732864009
FOLD: 2, EPOCH: 45, train_loss: 0.01890081726014614
FOLD: 2, EPOCH: 45, valid_loss: 0.017358416174021032
FOLD: 2, EPOCH: 46, train_loss: 0.018791468982254304
FOLD: 2, EPOCH: 46, valid_loss: 0.017228804270012513
FOLD: 2, EPOCH: 47, train_loss: 0.01863703273477093


FOLD: 3, EPOCH: 57, train_loss: 0.017188975281052052
FOLD: 3, EPOCH: 57, valid_loss: 0.01678875730269485
FOLD: 3, EPOCH: 58, train_loss: 0.017122101092771177
FOLD: 3, EPOCH: 58, valid_loss: 0.01680145924910903
FOLD: 3, EPOCH: 59, train_loss: 0.017103214543913642
FOLD: 3, EPOCH: 59, valid_loss: 0.01679219202035003
FOLD: 4, EPOCH: 0, train_loss: 0.6769494147069992
FOLD: 4, EPOCH: 0, valid_loss: 0.38718157675531173
FOLD: 4, EPOCH: 1, train_loss: 0.06167349284214358
FOLD: 4, EPOCH: 1, valid_loss: 0.020256776155696973
FOLD: 4, EPOCH: 2, train_loss: 0.023346074622484947
FOLD: 4, EPOCH: 2, valid_loss: 0.01851510453141398
FOLD: 4, EPOCH: 3, train_loss: 0.023229821963656333
FOLD: 4, EPOCH: 3, valid_loss: 0.018103088873128097
FOLD: 4, EPOCH: 4, train_loss: 0.021204670302329524
FOLD: 4, EPOCH: 4, valid_loss: 0.01732971239835024
FOLD: 4, EPOCH: 5, train_loss: 0.020915964954803065
FOLD: 4, EPOCH: 5, valid_loss: 0.3760144743654463
FOLD: 4, EPOCH: 6, train_loss: 0.023794447955104613
FOLD: 4, EPOCH: 6

FOLD: 5, EPOCH: 16, train_loss: 0.02068146486436167
FOLD: 5, EPOCH: 16, valid_loss: 0.01949269448717435
FOLD: 5, EPOCH: 17, train_loss: 0.020765244420016964
FOLD: 5, EPOCH: 17, valid_loss: 0.0187286204761929
FOLD: 5, EPOCH: 18, train_loss: 0.020798111959330498
FOLD: 5, EPOCH: 18, valid_loss: 0.01919536478817463
FOLD: 5, EPOCH: 19, train_loss: 0.02080445304032295
FOLD: 5, EPOCH: 19, valid_loss: 0.01879189329014884
FOLD: 5, EPOCH: 20, train_loss: 0.0207643372397269
FOLD: 5, EPOCH: 20, valid_loss: 0.01867556261519591
FOLD: 5, EPOCH: 21, train_loss: 0.02077473644768038
FOLD: 5, EPOCH: 21, valid_loss: 0.019319089853929147
FOLD: 5, EPOCH: 22, train_loss: 0.020743354319805098
FOLD: 5, EPOCH: 22, valid_loss: 0.018653179570618603
FOLD: 5, EPOCH: 23, train_loss: 0.020849509849663703
FOLD: 5, EPOCH: 23, valid_loss: 0.01864920411672857
FOLD: 5, EPOCH: 24, train_loss: 0.020729336953691897
FOLD: 5, EPOCH: 24, valid_loss: 0.018500335204104584
FOLD: 5, EPOCH: 25, train_loss: 0.020803929853343194
FOLD:

...

FOLD: 5, EPOCH: 16, train_loss: 0.02092865687464514
FOLD: 5, EPOCH: 16, valid_loss: 0.018770918974445924
FOLD: 5, EPOCH: 17, train_loss: 0.0210078073725585
FOLD: 5, EPOCH: 17, valid_loss: 0.018478364269766543
FOLD: 5, EPOCH: 18, train_loss: 0.020990063214013654
FOLD: 5, EPOCH: 18, valid_loss: 0.018853151239454746
FOLD: 5, EPOCH: 19, train_loss: 0.02093133970854744
FOLD: 5, EPOCH: 19, valid_loss: 0.01898873969912529
FOLD: 5, EPOCH: 20, train_loss: 0.021019159473719136
FOLD: 5, EPOCH: 20, valid_loss: 0.019724585426350433
FOLD: 5, EPOCH: 21, train_loss: 0.021190430344112457
FOLD: 5, EPOCH: 21, valid_loss: 0.018990802475147776
FOLD: 5, EPOCH: 22, train_loss: 0.02098339848941372
FOLD: 5, EPOCH: 22, valid_loss: 0.01874460776646932
FOLD: 5, EPOCH: 23, train_loss: 0.020915364882638378
FOLD: 5, EPOCH: 23, valid_loss: 0.019151796793772116
FOLD: 5, EPOCH: 24, train_loss: 0.021015117930308466
FOLD: 5, EPOCH: 24, valid_loss: 0.018829929642379284
FOLD: 5, EPOCH: 25, train_loss: 0.02096984880345483
F

FOLD: 6, EPOCH: 35, train_loss: 0.020064581726347248
FOLD: 6, EPOCH: 35, valid_loss: 0.018386119976639748
FOLD: 6, EPOCH: 36, train_loss: 0.01989614419158428
FOLD: 6, EPOCH: 36, valid_loss: 0.018207470612490877
FOLD: 6, EPOCH: 37, train_loss: 0.019861410389984808
FOLD: 6, EPOCH: 37, valid_loss: 0.018443097853485274
FOLD: 6, EPOCH: 38, train_loss: 0.01977636608625612
FOLD: 6, EPOCH: 38, valid_loss: 0.01817048719043241
FOLD: 6, EPOCH: 39, train_loss: 0.019615908375670833
FOLD: 6, EPOCH: 39, valid_loss: 0.01808034579324372
FOLD: 6, EPOCH: 40, train_loss: 0.01950929360764642
FOLD: 6, EPOCH: 40, valid_loss: 0.01812888917458408
FOLD: 6, EPOCH: 41, train_loss: 0.019377455812308098
FOLD: 6, EPOCH: 41, valid_loss: 0.018161394578569075
FOLD: 6, EPOCH: 42, train_loss: 0.01923542277466866
FOLD: 6, EPOCH: 42, valid_loss: 0.018013137721401805
FOLD: 6, EPOCH: 43, train_loss: 0.01912347345102218
FOLD: 6, EPOCH: 43, valid_loss: 0.017906326910152155
FOLD: 6, EPOCH: 44, train_loss: 0.01892851023904739
FO

FOLD: 7, EPOCH: 54, train_loss: 0.017473405942080484
FOLD: 7, EPOCH: 54, valid_loss: 0.017113897730322444
FOLD: 7, EPOCH: 55, train_loss: 0.017310734922366757
FOLD: 7, EPOCH: 55, valid_loss: 0.01709349869805224
FOLD: 7, EPOCH: 56, train_loss: 0.01723312897187087
FOLD: 7, EPOCH: 56, valid_loss: 0.017097363631953213
FOLD: 7, EPOCH: 57, train_loss: 0.017152420233093924
FOLD: 7, EPOCH: 57, valid_loss: 0.017105504213010565
FOLD: 7, EPOCH: 58, train_loss: 0.017107765809182197
FOLD: 7, EPOCH: 58, valid_loss: 0.017111860489582315
FOLD: 7, EPOCH: 59, train_loss: 0.01711736175441934
FOLD: 7, EPOCH: 59, valid_loss: 0.017120561014641735
FOLD: 8, EPOCH: 0, train_loss: 0.6758094708765706
FOLD: 8, EPOCH: 0, valid_loss: 0.3549552410840988
FOLD: 8, EPOCH: 1, train_loss: 0.06033824683437424
FOLD: 8, EPOCH: 1, valid_loss: 0.01983783430316382
FOLD: 8, EPOCH: 2, train_loss: 0.02318322795533365
FOLD: 8, EPOCH: 2, valid_loss: 0.01886819965309567
FOLD: 8, EPOCH: 3, train_loss: 0.022247767135981592
FOLD: 8, EP

FOLD: 9, EPOCH: 13, train_loss: 0.020675271172677315
FOLD: 9, EPOCH: 13, valid_loss: 0.01786959802524911
FOLD: 9, EPOCH: 14, train_loss: 0.020669905769248164
FOLD: 9, EPOCH: 14, valid_loss: 0.01859853251112832
FOLD: 9, EPOCH: 15, train_loss: 0.020673594671872353
FOLD: 9, EPOCH: 15, valid_loss: 0.017988905724551942
FOLD: 9, EPOCH: 16, train_loss: 0.021014503293460416
FOLD: 9, EPOCH: 16, valid_loss: 0.022399566343261137
FOLD: 9, EPOCH: 17, train_loss: 0.021344929536984813
FOLD: 9, EPOCH: 17, valid_loss: 0.019270812678668234
FOLD: 9, EPOCH: 18, train_loss: 0.021560025888104593
FOLD: 9, EPOCH: 18, valid_loss: 0.019824980965091124
FOLD: 9, EPOCH: 19, train_loss: 0.021505530022325053
FOLD: 9, EPOCH: 19, valid_loss: 0.020638613340755303
FOLD: 9, EPOCH: 20, train_loss: 0.0212573014800587
FOLD: 9, EPOCH: 20, valid_loss: 0.019517432070440717
FOLD: 9, EPOCH: 21, train_loss: 0.021321231643519095
FOLD: 9, EPOCH: 21, valid_loss: 0.018134994225369558
FOLD: 9, EPOCH: 22, train_loss: 0.0209825864241969

KeyError: "None of [Index(['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',\n       'acat_inhibitor', 'acetylcholine_receptor_agonist',\n       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',\n       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',\n       'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist',\n       ...\n       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',\n       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',\n       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',\n       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],\n      dtype='object', length=206)] are in the [columns]"

In [111]:

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(seed=42)

In [19]:
pred1 = np.load('moa_oof_01844.npy')
pred2 = np.load('model_oof_01840.npy')
pred3 = np.load('moa_oof_01858.npy')
pred4 = np.load('moa_oof_xgb.npy')
pred5 = np.load('model_newcv_oof_01835.npy')
pred6 = np.load('moa_oof_newcv_01844.npy')
pred7= np.load('moa_oof_newcv_01858_opt.npy')

pred8 = np.load('model_newcv_oof_01835.npy')
pred9 = np.load('moa_oof_newcv_01858_7folds.npy')
pred10 = np.load('moa_oof_newcv_01838_7folds.npy')
pred11 = np.load('moa_oof_newcv_01858.npy')
pred12 = np.load('moa_oof_newcv_01858_label.npy')
pred13 = np.load('moa_10folds.npy')
pred14 = np.load('model_newcv_oof_01838_5folds.npy')







In [33]:
import datetime
import pandas as pd
from time import time
# from autograd import grad
# import autograd.numpy as np
import numpy as np
from numba import njit
from scipy.optimize import minimize, fsolve

In [34]:
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
    return loss.mean()

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
    return log_loss_numpy(oof_blend)

def grad_func(weights):
    oof_clip = np.clip(oof, 1e-15, 1 - 1e-15)
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients
@njit(nopython=True)
def grad_func_jit(weights):
    oof_clip = np.minimum(1 - 1e-15, np.maximum(oof, 1e-15))
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

In [35]:
#y_true = pd.read_csv('D:\\Dataset\\MOA\\train_targets_scored.csv', index_col = 'sig_id').values

oof_dict = {'Model 1': 'model_newcv_oof_01835.npy', 
            'Model 2': 'moa_oof_newcv_01858_label.npy', 
            'Model 3': 'model_newcv_oof_01838_5folds.npy'
           }

oof = np.zeros((len(oof_dict), train.shape[0], len(target_cols)))
for i in range(oof.shape[0]):
    oof[i] = np.load(list(oof_dict.values())[i])

In [36]:
y_true = target.iloc[:, 1:].values

In [37]:
%%time

log_loss_scores = {}
for n, key in enumerate(oof_dict.keys()):
    score_oof = log_loss_numpy(oof[n])
    log_loss_scores[key] = score_oof
    print(f'{key} CV:\t', score_oof)
print('-' * 50)

Model 1 CV:	 0.017073878563896246
Model 2 CV:	 0.01713246054364053
Model 3 CV:	 0.017330495549518366
--------------------------------------------------
Wall time: 656 ms


In [38]:
test_weights = np.array([1 / oof.shape[0]] * oof.shape[0])

In [39]:
%timeit -r 10 grad_func(test_weights)


1.27 s ± 12.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [40]:
%timeit -r 10 grad_func_jit(test_weights)


306 ms ± 5.65 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [41]:
tol = 1e-10
init_guess = [1 / oof.shape[0]] * oof.shape[0]
bnds = [(0, 1) for _ in range(oof.shape[0])]
cons = {'type': 'eq', 
        'fun': lambda x: np.sum(x) - 1, 
        'jac': lambda x: [1] * len(x)}

print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()
res_scipy = minimize(fun = func_numpy_metric, 
                     x0 = init_guess, 
                     method = 'SLSQP', 
                     jac = grad_func_jit, # grad_func 
                     bounds = bnds, 
                     constraints = cons, 
                     tol = tol)
print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
print('Optimised Weights:', res_scipy.x)

Inital Blend OOF: 0.017019816066787135
[00:11] Optimised Blend OOF: 0.016994478723817687
Optimised Weights: [0.52035188 0.41286638 0.06678174]


In [42]:
print('Check the sum of all weights:', np.sum(res_scipy.x))
if np.sum(res_scipy.x) - 1 <= tol:
    print('Great! The sum of all weights equals to 1!')
else:
    print('Manual adjustion is needed to modify the weights.')

Check the sum of all weights: 1.0
Great! The sum of all weights equals to 1!


In [31]:
train[target_cols] = (pred8 * 0.45728174 + pred12 * 0.27524878 + pred1 * 0.26746948)

In [86]:
train[target_cols] = ((pred8 * 0.45728174 + pred12 * 0.27524878 + pred1 * 0.26746948) * 5 + (pred13))/6

In [43]:
train[target_cols] = (pred8 * 5 + pred12 * 4 + pred14)/10

In [137]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)



y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.015594240552897484


In [79]:
pred1 = np.load('moa_01844.npy')
pred2 = np.load('model_newcv_01835.npy')
pred3 = np.load('moa_01844.npy')
pred4 = np.load('model_01840.npy')
pred5 = np.load('moa_01858.npy')
pred6 = np.load('moa_xgb.npy')
pred7 = np.load('moa_newcv_01844.npy')
#np.save('moa_oof_newcv_01858',oof)

pred8 = np.load('moa_newcv_01858.npy')

pred9 = np.load('model_newcv_01835_7folds.npy')
pred10 = np.load('model_newcv_01835.npy')
pred11 = np.load('moa_newcv_01858_7folds.npy')
pred12 = np.load('moa_newcv_01858_label.npy')
pred13 = np.load('moa_10folds_preds_1.npy')



In [89]:
for i in target_cols:
    test[i]=0.

In [41]:
test[target_cols] = (pred10 * 0.45728174 + pred12 * 0.27524878 + pred1 * 0.26746948)

In [90]:
test[target_cols] = ((pred10 * 0.45728174 + pred12 * 0.27524878 + pred1 * 0.26746948) *5 + (pred13))/6

In [91]:
sample_submission = pd.read_csv('D:\\Dataset\\MOA\\sample_submission.csv')

In [92]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission_24_11_2020_v2.csv', index=False)

In [82]:
oof

array([[0.00103485, 0.00147651, 0.00094714, ..., 0.00162948, 0.00064708,
        0.00183387],
       [0.00094011, 0.0006696 , 0.00304768, ..., 0.0045382 , 0.00178554,
        0.00376633],
       [0.00163513, 0.00370731, 0.00093928, ..., 0.00069495, 0.00061159,
        0.00132486],
       ...,
       [0.00070007, 0.00109894, 0.00218864, ..., 0.00166229, 0.00682262,
        0.00189743],
       [0.00068031, 0.00054134, 0.00110733, ..., 0.0004013 , 0.00048773,
        0.00112408],
       [0.0004891 , 0.00121476, 0.00145994, ..., 0.00062922, 0.00065504,
        0.00024006]])

In [42]:
np.save('moa_oof_newcv_01858_label',oof)
np.save('moa_newcv_01858_label',predictions)